In [40]:
!pip install -qU langchain-redis langchain-openai redis

In [41]:
!docker run -d  -p 6379:6379 redislabs/redisearch:latest

94e6d5c9742ae6accfd7b12f07cf8c166aed0dfb7485eaf25747f7a34eb1ef51
docker: Error response from daemon: driver failed programming external connectivity on endpoint jolly_satoshi (014da900f1cece31199fb0f87bd8aef91c97d302bc569a5114007dbd97efd078): Bind for 0.0.0.0:6379 failed: port is already allocated.


In [42]:
import os

# Use the environment variable if set, otherwise default to localhost
REDIS_URL = os.getenv("REDIS_URL", "redis://localhost:6379")
print(f"Connecting to Redis at: {REDIS_URL}")




Connecting to Redis at: redis://localhost:6379


In [43]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_redis import RedisChatMessageHistory

In [44]:
history = RedisChatMessageHistory(session_id="user_123", redis_url=REDIS_URL)

history.add_user_message("Hello, AI Assistant!")
history.add_ai_message("Hello! How can I assist you today?")

print("Chat History:")

for message in history.messages:
    print(f"{type(message).__name__}: {message.content}")

Chat History:
HumanMessage: Hello, AI Assistant!
AIMessage: Hello! How can I assist you today?
HumanMessage: Hello, AI Assistant!
AIMessage: Hello! How can I assist you today?


In [45]:
from getpass import getpass

# Check if OPENAI_API_KEY is already set in the environment
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    print("OpenAI API key not found in environment variables.")
    openai_api_key = getpass("Please enter your OpenAI API key: ")

    # Set the API key for the current session
    os.environ["OPENAI_API_KEY"] = openai_api_key
    print("OpenAI API key has been set for this session.")
else:
    print("OpenAI API key found in environment variables.")

OpenAI API key found in environment variables.


In [64]:

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
    
)

llm = ChatOpenAI()

chain = prompt | llm

def get_redis_message_history(session_id: str) -> BaseChatMessageHistory:
    return RedisChatMessageHistory(session_id, REDIS_URL)

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_redis_message_history,
    input_messages_key="input",
    history_messages_key="history"
)


response1 = chain_with_history.invoke({"input": "Multiply 2 by 3"}, config={"session_id": "alice_123"})

print("AI Response 1:", response1.content)

response2 = chain_with_history.invoke(
    {"input": "Then multiply with 5"}, config={"session_id": "alice_123"}
)
print("AI Response 2:", response2.content)



AI Response 1: 2 multiplied by 3 is equal to 6.
AI Response 2: If you multiply 6 by 5, you get 30.
